In [107]:
#Initialize SparkSession and SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
#from pyspark import SparkContext

#Create a Spark Session
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Test-spark") \
    .config("spark.executor.memory", "6g") \
    .config("spark.cores.max","4") \
    .getOrCreate()


#Get the Spark Context from Spark Session    
sc = spark.sparkContext

In [108]:
sc

<SparkContext master=local[*] appName=Test-spark>

In [109]:
sc.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.app.name', 'Test-spark'),
 ('spark.app.id', 'local-1676641408088'),
 ('spark.driver.port', '50211'),
 ('spark.executor.id', 'driver'),
 ('spark.app.submitTime', '1676637870756'),
 ('spark.cores.max', '4'),
 ('spark.executor.

-- UNDERSTAT --

In [110]:
myDF1 = spark.read.format('csv') \
.option('inferSchema', 'true') \
.option('delimiter', ',') \
.option('header','true') \
.load('../webscrapingtfm/Stats_Understat.csv')
myDF1bis = spark.read.format('csv') \
.option('inferSchema', 'true') \
.option('delimiter', ',') \
.option('header','true') \
.load('../webscrapingtfm/Stats_Understat_18_19.csv')

myDF1 = myDF1.union(myDF1bis)
myDF1.createOrReplaceTempView("UnderstatTable")

In [111]:
spark.sql("SELECT * FROM UnderstatTable").show()

+---+---------+----+---+---+---+---+----------+----------+-----+----+-----+----+---------------+
| Nº|Formation| Min| Sh|  G|ShA| GA|        xG|       xGA|  xGD|xG90|xGA90| Año|           Club|
+---+---------+----+---+---+---+---+----------+----------+-----+----+-----+----+---------------+
|  1|    4-3-3|1458|256| 31|115|  5|40.30+9.30|12.15+7.15|28.15|2.49| 0.75|2022|      Barcelona|
|  2|  3-2-4-1|  76| 14|  3|  9|  1| 1.48-1.52| 0.78-0.22| 0.71|1.76| 0.92|2022|      Barcelona|
|  3|  4-4-1-1|  64|  6|  2| 10|  0| 0.84-1.16| 0.91+0.91|-0.07|1.18| 1.28|2022|      Barcelona|
|  4|  3-1-4-2|  12|  0|  0|  2|  0|      0.00| 0.10+0.10| -0.1| 0.0| 0.77|2022|      Barcelona|
|  1|    4-3-3|1605|277| 38|199| 16|36.91-1.09|19.23+3.23|17.67|2.07| 1.08|2022|    Real_Madrid|
|  2|  4-2-3-1|  16|  6|  0|  1|  0| 0.60+0.60| 0.06+0.06| 0.55| 3.4| 0.32|2022|    Real_Madrid|
|  3|    3-4-3|   7|  3|  0|  1|  0| 0.66+0.66| 0.01+0.01| 0.64|8.46| 0.19|2022|    Real_Madrid|
|  1|  4-1-3-2| 760|108| 11| 7

Formaciones en el fútbol puede haber tantas como uno quiera, pero hay algunas que tan solo son pequeñas variantes de otras.
Por tanto, para simplificar el análisis vamos a quedarnos únicamente con las formaciones primarias.

In [112]:
spark.sql("SELECT COUNT(DISTINCT(Formation)) FROM UnderstatTable").show()
spark.sql("SELECT DISTINCT(Formation) FROM UnderstatTable").show(22)

+-------------------------+
|count(DISTINCT Formation)|
+-------------------------+
|                       22|
+-------------------------+

+---------+
|Formation|
+---------+
|    4-5-1|
|  4-2-4-0|
|    5-4-1|
|    3-5-2|
|  3-3-3-1|
|  3-4-2-1|
|  4-1-3-2|
|    4-3-3|
|  4-2-2-2|
|    3-4-3|
|    5-3-2|
|  3-4-1-2|
|  3-5-1-1|
|  4-2-3-1|
|4-1-2-1-2|
|  4-3-2-1|
|    4-4-2|
|  3-2-4-1|
|  4-3-1-2|
|  3-1-4-2|
|  4-1-4-1|
|  4-4-1-1|
+---------+



In [113]:
DFUnderstat = myDF1
DFUnderstat = myDF1.drop('Nº', 'xG', 'xGA', 'xGD', 'xG90', 'xGA90') \
                            .withColumn('Club',f.regexp_replace('Club','_',' ')) \
                            .withColumn('Formation', f.when(DFUnderstat.Formation == "4-2-4-0","4-4-2")
                                .when(DFUnderstat.Formation == "3-3-3-1","3-4-3")
                                .when(DFUnderstat.Formation == "4-2-2-2","4-4-2")
                                .when(DFUnderstat.Formation == "3-4-2-1","3-4-3")
                                .when(DFUnderstat.Formation == "3-4-1-2","3-5-2")
                                .when(DFUnderstat.Formation == "3-5-1-1","3-4-3")
                                .when(DFUnderstat.Formation == "4-1-2-1-2","4-3-1-2")
                                .when(DFUnderstat.Formation == "4-1-3-2","4-3-1-2")
                                .when(DFUnderstat.Formation == "4-3-2-1","4-3-3")
                                .when(DFUnderstat.Formation == "3-2-4-1","3-4-3")
                                .when(DFUnderstat.Formation == "3-1-4-2","3-5-2")
                                .when(DFUnderstat.Formation == "4-1-4-1","4-3-3")
                                .when(DFUnderstat.Formation == "4-4-1-1","4-5-1")
                                .otherwise(DFUnderstat.Formation)) \
                            .groupBy('Club','Año','Formation') \
                            .agg(f.sum('Min'),f.sum('Sh'),f.sum('G'),f.sum('ShA'),f.sum('GA')) \
                            .orderBy(f.desc('Año'),'Club',f.desc('sum(Min)'))
DFUnderstat.show()

+--------+----+---------+--------+-------+------+--------+-------+
|    Club| Año|Formation|sum(Min)|sum(Sh)|sum(G)|sum(ShA)|sum(GA)|
+--------+----+---------+--------+-------+------+--------+-------+
|AC Milan|2022|  4-2-3-1|    1489|    248|    31|     183|     22|
|AC Milan|2022|    4-3-3|     103|     21|     2|      11|      0|
|AC Milan|2022|    5-3-2|      81|     10|     1|      11|      2|
|AC Milan|2022|    3-5-2|      60|     13|     0|       2|      0|
|AC Milan|2022|    4-4-2|      37|      6|     1|       2|      0|
|AC Milan|2022|    3-4-3|      35|      3|     0|       4|      0|
|AC Milan|2022|    5-4-1|      10|      2|     0|       4|      0|
| Ajaccio|2022|    4-4-2|    1629|    181|    15|     186|     26|
| Ajaccio|2022|    4-3-3|      86|      3|     0|       9|      3|
| Ajaccio|2022|    5-4-1|      48|      2|     1|       6|      5|
| Ajaccio|2022|    4-5-1|      18|      0|     0|       5|      0|
| Almeria|2022|    4-3-3|     863|    118|    12|     104|    

In [114]:
#DFUnderstat.where(DFUnderstat['Año'] == '2020').count()
#DFUnderstat.where(DFUnderstat['Año'] == '2021').count()
DFUnderstat.where(DFUnderstat['Año'] == '2022').count()

487

-- TRANSFERMARKT --

In [115]:
myDF2 = spark.read.format('csv') \
.option('inferSchema', 'true') \
.option('delimiter', ',') \
.option('header','true') \
.load('../webscrapingtfm/Stats_Transfermarktt_clean_sep.csv')
myDF2bis = spark.read.format('csv') \
.option('inferSchema', 'true') \
.option('delimiter', ',') \
.option('header','true') \
.load('../webscrapingtfm/Stats_Transfermarktt_18_19_corr_SEP.csv')

myDF2 = myDF2.union(myDF2bis)
myDF2.createOrReplaceTempView("TransfermarktTable")

In [116]:
spark.sql("SELECT * FROM TransfermarktTable").show()

+---------------+------------+----+-----------+-------------------+----------------+----+------+
|           Club|Nº jugadores|Edad|Extranjeros|Media valor mercado|Valor de mercado| Año|  Liga|
+---------------+------------+----+-----------+-------------------+----------------+----+------+
|      Barcelona|          32|25.1|         20|      25.42 mill. €|  813.50 mill. €|2020|laliga|
|    Real Madrid|          37|24.8|         17|      21.28 mill. €|  787.30 mill. €|2020|laliga|
|Atletico Madrid|          39|24.9|         21|      19.82 mill. €|  773.10 mill. €|2020|laliga|
|        Sevilla|          38|25.6|         17|      11.01 mill. €|  418.25 mill. €|2020|laliga|
|  Real Sociedad|          38|24.7|          6|      10.70 mill. €|  406.60 mill. €|2020|laliga|
|     Villarreal|          36|25.9|         13|       8.13 mill. €|  292.60 mill. €|2020|laliga|
|       Valencia|          35|24.7|         20|       7.97 mill. €|  278.85 mill. €|2020|laliga|
|     Real Betis|          33|

In [117]:
DFTransfermarkt = myDF2
DFTransfermarkt = myDF2.drop('Nº jugadores', 'Extranjeros', 'Media valor mercado') \
                        .withColumn('Valor de mercado',f.regexp_replace('Valor de mercado',' mil mill. €','')) \
                        .withColumn('Valor de mercado',f.regexp_replace('Valor de mercado',' mill. €','')) \
                        .withColumnRenamed('Valor de mercado','Valor plantilla (mill. €)')
DFTransfermarkt.show()

+---------------+----+-------------------------+----+------+
|           Club|Edad|Valor plantilla (mill. €)| Año|  Liga|
+---------------+----+-------------------------+----+------+
|      Barcelona|25.1|                   813.50|2020|laliga|
|    Real Madrid|24.8|                   787.30|2020|laliga|
|Atletico Madrid|24.9|                   773.10|2020|laliga|
|        Sevilla|25.6|                   418.25|2020|laliga|
|  Real Sociedad|24.7|                   406.60|2020|laliga|
|     Villarreal|25.9|                   292.60|2020|laliga|
|       Valencia|24.7|                   278.85|2020|laliga|
|     Real Betis|26.8|                   207.70|2020|laliga|
|  Athletic Club|25.9|                   204.15|2020|laliga|
|         Getafe|25.4|                   170.10|2020|laliga|
|     Celta Vigo|24.3|                   155.85|2020|laliga|
|        Granada|24.9|                   122.65|2020|laliga|
|        Levante|26.5|                   113.00|2020|laliga|
|        Osasuna|27.0|  

-- WHOSCORED --

In [118]:
myDF3 = spark.read.format('csv') \
.option('inferSchema', 'true') \
.option('delimiter', ',') \
.option('header','true') \
.load('../webscrapingtfm/Stats_WhoScored_clean_sep.csv')
myDF3bis = spark.read.format('csv') \
.option('inferSchema', 'true') \
.option('delimiter', ',') \
.option('header','true') \
.load('../webscrapingtfm/Stats_WhoScored_18_19_corr_SEP.csv')

myDF3 = myDF3.union(myDF3bis)
myDF3.createOrReplaceTempView("WhoScoredTable")

In [119]:
spark.sql("SELECT * FROM WhoScoredTable").show()

+---------------+-----+--------+---------+-----+------------+----------------+------+------+----+-----------+
|           Club|Goles|Tiros pp|Amarillas|Rojas|Posesión (%)|Acierto Pase (%)|Aéreos|Rating| Año|       Pais|
+---------------+-----+--------+---------+-----+------------+----------------+------+------+----+-----------+
|      Barcelona|   85|    15.3|       68|    2|        36.9|            89.7|  10.6|  6.87|2020|Espa%C3%B1a|
|    Real Madrid|   67|    14.4|       57|    2|        57.7|            87.7|  11.8|  6.86|2020|Espa%C3%B1a|
|Atletico Madrid|   67|    12.1|      100|    0|        51.8|            83.1|  14.4|  6.84|2020|Espa%C3%B1a|
|        Sevilla|   53|    12.1|       75|    2|        58.7|            86.2|  16.6|   6.7|2020|Espa%C3%B1a|
|     Villarreal|   60|    10.7|       65|    5|        54.3|            84.4|  13.0|  6.66|2020|Espa%C3%B1a|
|  Real Sociedad|   59|    11.3|       81|    1|        53.7|            80.8|  17.9|  6.65|2020|Espa%C3%B1a|
|     Real

In [120]:
DFWhoScored = myDF3
DFWhoScored = myDF3.drop('Goles', 'Tiros pp', 'Aéreos') \
                            .withColumn('Pais',f.regexp_replace('Pais','Espa%C3%B1a','España'))
DFWhoScored.show()

+---------------+---------+-----+------------+----------------+------+----+------+
|           Club|Amarillas|Rojas|Posesión (%)|Acierto Pase (%)|Rating| Año|  Pais|
+---------------+---------+-----+------------+----------------+------+----+------+
|      Barcelona|       68|    2|        36.9|            89.7|  6.87|2020|España|
|    Real Madrid|       57|    2|        57.7|            87.7|  6.86|2020|España|
|Atletico Madrid|      100|    0|        51.8|            83.1|  6.84|2020|España|
|        Sevilla|       75|    2|        58.7|            86.2|   6.7|2020|España|
|     Villarreal|       65|    5|        54.3|            84.4|  6.66|2020|España|
|  Real Sociedad|       81|    1|        53.7|            80.8|  6.65|2020|España|
|     Real Betis|       87|    8|        52.9|            82.0|  6.64|2020|España|
|       Valencia|       77|    5|        47.9|            79.4|   6.6|2020|España|
|     Celta Vigo|      104|    5|        52.0|            79.9|   6.6|2020|España|
|   

UNION DE ARCHIVOS

In [121]:
DFGlobal = DFUnderstat.join(DFTransfermarkt,['Club','Año']) \
                    .join(DFWhoScored,['Club','Año']) \
                    .drop('Liga') \
                    .orderBy(f.desc('Año'),'Club',f.desc('sum(Min)'))
DFGlobal.show()

+--------+----+---------+--------+-------+------+--------+-------+----+-------------------------+---------+-----+------------+----------------+------+-------+
|    Club| Año|Formation|sum(Min)|sum(Sh)|sum(G)|sum(ShA)|sum(GA)|Edad|Valor plantilla (mill. €)|Amarillas|Rojas|Posesión (%)|Acierto Pase (%)|Rating|   Pais|
+--------+----+---------+--------+-------+------+--------+-------+----+-------------------------+---------+-----+------------+----------------+------+-------+
|AC Milan|2022|  4-2-3-1|    1489|    248|    31|     183|     22|26.3|                   574.40|       50|    2|        54.8|            81.2|  6.66| Italia|
|AC Milan|2022|    4-3-3|     103|     21|     2|      11|      0|26.3|                   574.40|       50|    2|        54.8|            81.2|  6.66| Italia|
|AC Milan|2022|    5-3-2|      81|     10|     1|      11|      2|26.3|                   574.40|       50|    2|        54.8|            81.2|  6.66| Italia|
|AC Milan|2022|    3-5-2|      60|     13|    

In [122]:
DFGlobal.write.csv('Stats_Global_18_22',header = True)

In [123]:
sc.stop()